In [3]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from selenium.common.exceptions import *


In [7]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(2)

    wait = WebDriverWait(driver, 2)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()
    time.sleep(2)
#   while True:
    scroll_height = 10
    document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
    driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
    time.sleep(1)
#   document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
#       if document_height_after == document_height_before:
#            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [8]:
df = search_youtube_videos('Ironhack Barcelona')

In [9]:
df

,Name,views,duration,date,description,link
0,Discover Ironhack Barcelona,"79,425","1 minute, 16 seconds",4 years ago,This is what Ironhack Barcelona looks like ▻ F...,https://www.youtube.com/watch?v=s2cv8AV-lFs
1,Ironhack Barcelona UX UI Design Part Time,656,"3 minutes, 3 seconds",1 year ago,"In this video, Edu Huerta (Lead Teacher of UX ...",https://www.youtube.com/watch?v=UdSKUE17RiM
2,Ironhack Barcelona: Coding Bootcamp Student Gr...,unknown,Unknown,5 years ago,Multistreaming with https://restream.io/ ▻ Sub...,https://www.youtube.com/watch?v=9K2GpMIoscU
3,Ironhack Barcelona: Coding Bootcamp Student Gr...,unknown,Unknown,5 years ago,Multistreaming with https://restream.io/ ▻ Sub...,https://www.youtube.com/watch?v=pO8UgMs01YI
4,Un año después de hacer el Bootcamp de Ironhac...,"8,464","11 minutes, 35 seconds",2 years ago,Muy buenas Velociraptores! Después de hablar c...,https://www.youtube.com/watch?v=XsY3Pv3Al6A


In [10]:
def scrape_comments(df):
    comments = []

    for i, row in df.iterrows():
        url = row['link']
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)
        try:
            wait = WebDriverWait(driver, 10)
            cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
            cookie_consent.click()
        except TimeoutException:
            print('Cookie pop-up not loaded')

        time.sleep(5)
        
        driver.execute_script("window.scrollBy(0, 500);")

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, '//ytd-comment-thread-renderer')))
            time.sleep(30)
            while True:
                scroll_height = 10
                document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
                driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
                time.sleep(2)
                document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
                if document_height_after == document_height_before:
                    break

            html = driver.page_source
            try:
                soup = BeautifulSoup(html, "html.parser")

                for comment in soup.find_all('ytd-comment-thread-renderer'):
                    text = comment.find('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'}).text
                    likes = comment.find('span', {'id': 'vote-count-left'}).text.strip() if comment.find('span', {'id': 'vote-count-left'}) else 0
                    replies = comment.find('button', {'class': 'yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text'})
                    if replies:
                        num_comments = replies.get('aria-label').split()[0]
                    else:
                        num_comments = 0
                    username_element = comment.find('a', {'id': 'author-text'}).find('span', {'class': 'style-scope ytd-comment-renderer'})
                    if username_element:
                        username = username_element.text.replace('[','').replace(']','').replace('\n','')
                    else:
                        username = ''
                    link = url
                    try:
                        date = comment.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string')
                    except:
                        date = 'Unknown'
            except:
                print('Fail')

                comments.append([username, text, likes, num_comments, date, link])
        except TimeoutException:
            comments.append(["ERROR", 'ERROR', 'ERROR', 'ERROR', 'ERROR', url])
        finally:
            driver.quit()
            print(f'{url} done')

        df_comments = pd.DataFrame(comments, columns=['Username', 'Comment', 'Likes', 'Replies', 'Date', 'link'])

    return df_comments


In [11]:
scrape_comments(df)

https://www.youtube.com/watch?v=s2cv8AV-lFs done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=UdSKUE17RiM done


KeyboardInterrupt: 